# Supervised learning - regression

In supervised regression, we want to train the algorithm to predict a value, rather than a categorical value as do in classification. These techniques are essentially the same as regression in statistics.

In this notebook we will look at how to create a regression pipeline using the tools available in the Scikit-learn library. The examples are quite simple, but demonstrate the key steps in doing regression. The inerface for each type of class in the process is quite consistent for all the different examples of that type, so een when dealing with much  more complicated examples, the interface will be the same and so the code will look similar. See the scikit-learn documentation for more information on the functions used:

* user Guide: https://scikit-learn.org/stable/user_guide.html
* API reference: https://scikit-learn.org/stable/modules/classes.html

In [1]:
import os
import numpy
import pandas
import matplotlib
import matplotlib.pyplot
import mpl_toolkits.mplot3d

In [2]:
import sklearn
import sklearn.datasets
import sklearn.model_selection
import sklearn.preprocessing
import sklearn.linear_model
import sklearn.metrics
import sklearn.neural_network
import sklearn.kernel_ridge

## Data loading and exploration
We will use a generated dataset from scikit-learn to demonstrate the regression pipeline. To better demonstrate a standard pipeline, I am changing the mean and standard deviation of the input features to represent a dataset closer to a real example where preprpocessing is required.

Documentation:
* https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html#sklearn.datasets.make_regression

In [3]:
n_samples=10000
n_features=10
n_informative=5
n_targets = 1
noise_std = 0.2

In [4]:
scale_random = numpy.repeat(numpy.random.random([1,n_features])*5, n_samples, axis=0)
translate_random = numpy.repeat(numpy.array(numpy.random.random([1,n_features])*10-5,dtype=numpy.int32), n_samples, axis=0)
input_features, target_feature, coef1 =  sklearn.datasets.make_regression(n_samples=n_samples, 
                                         n_features=n_features, 
                                         n_informative=n_informative, 
                                         n_targets=n_targets, 
                                         noise=noise_std,
                                         coef=True)
input_features = (input_features * scale_random) + translate_random

In [5]:
coef1

array([23.0546468 , 95.35696524, 68.52221188,  0.        , 81.55822383,
        0.        ,  0.        ,  0.        , 53.7647458 ,  0.        ])

In [6]:
input_features.mean(axis=0)

array([ 1.99072483e+00, -9.28893139e-01,  9.83712094e-01,  5.13038590e-05,
       -1.00197827e+00,  9.75273201e-01, -2.02280857e+00, -9.92460508e-01,
       -3.29387352e-02,  3.01251571e+00])

In [7]:
input_features.std(axis=0)

array([1.03047592, 4.67366624, 1.85930993, 0.06850683, 0.55565249,
       4.24178359, 1.891661  , 1.57531091, 3.25687822, 1.11055211])

In [8]:
print(f'target variable range=[{target_feature.min()},{target_feature.max()}] mean={target_feature.mean()} std={target_feature.std()}')

target variable range=[-584.6791226261241,595.2368001496696] mean=-0.19857433154941817 std=155.4434495458798


## Data preprocessing

Before applying a supervised learning algorithm to our data, we usually have to do some preprocessing to get it into a suitable state for training. The most common is to normalise the data. This is so the algorthm gives equal importance to each of the features, which is a usual starting assumption when applying machine learning techniques.

Documentation:
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

Before we normalise the data, we should split our data into training and test sets. To preoperly evaulate our regressor performance, we must evaluate it on unseen data to see how well it generalises. We don't want any part of the pipeline to be trained on the data to be used for testing, so we do the split as early as possible.

In [9]:
input_train, input_test, target_train, target_test = sklearn.model_selection.train_test_split(input_features, target_feature)

For this data, to normaise the input features, we want each feature to have zero mean and standard deviation of 1. The StandardScaler class does this. There are many other preprocessing classes for differents sort of data and different required preprocessing.

In [10]:
input_scaler = sklearn.preprocessing.StandardScaler()
input_scaler.fit(input_train)
X_train = input_scaler.transform(input_train)
X_test = input_scaler.transform(input_test)
y_train = target_train
y_test = target_test

We can now see that the mean of each column of data is nearly zero, and the standard deviation is 1. our features are now ready for training.

In [11]:
X_train.mean(axis=0)

array([-2.23786730e-15, -9.28842188e-16,  3.33432541e-15, -5.04781402e-17,
       -1.15841411e-15, -6.69427476e-16, -3.93449717e-15,  1.65282602e-15,
        5.61920880e-17, -6.31124782e-15])

In [12]:
X_train.std(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Training regression algorithms

The next step is to train the  regression algorithm sing our data. We are using a linear regression so the training involves calculating the linear coefficients using an ordinary least squares solver, which in the case of scikit-learn is using the scipy implementation of this algorithm.

The regressor, as with most steps in a scikit-learn pipeline, have 2 key methods. For a regressor these are fit and predict. The call to fit uses the training data to calculate the parameters of the algorithm to give the best match to the target data. The call to predict then gives a prediction for each of the observations which will as close as possible to the known answer. One can then apply the algorithm to the unseen test data to how well the algorithm generalises for those parameters.

Documentation: 
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

Here we create a regressor object, then train the algorithm (i.e. calculate the parameters, in this case the linear coefficients). The calculated linear coefficients are displayed after fitting. Because we generated the data, we can compare our calculated coefficients to those used for generating the data. They are not an exact match because we added noise to our data to make it more "real".

In [13]:
linear_regressor1 = sklearn.linear_model.LinearRegression()
linear_regressor1.fit(X_train, y_train)
linear_regressor1.coef_

array([ 2.30722756e+01,  9.50878969e+01,  6.90344282e+01,  1.12411814e-03,
        8.13527520e+01,  1.49990169e-03,  3.93610507e-04, -2.19477198e-03,
        5.31601990e+01,  1.47116270e-03])

In [14]:
linear_regressor1.coef_ - coef1

array([ 1.76288299e-02, -2.69068291e-01,  5.12216318e-01,  1.12411814e-03,
       -2.05471822e-01,  1.49990169e-03,  3.93610507e-04, -2.19477198e-03,
       -6.04546764e-01,  1.47116270e-03])

Now that we've trained our algorithm, we use to calcuate results for both the observation used in training, so how good our fit it, and also on the unseen observations, to see how well the trained algorithm generalises.

In [15]:
y_out_train = linear_regressor1.predict(X_train)

In [16]:
y_out_test = linear_regressor1.predict(X_test)

## Performance metrics
The last step is to evaluate the performance of the trained algorithms. There are many different metrics available from which one choose based on the nature of the problem. They allhave the same interface, where one passes the true target values and the predicted target values and the results are calculated. In this case we are using the root mean squared error to measure the regression fit.

Documentation:
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html

In [17]:
err_train = sklearn.metrics.mean_squared_error(y_train, y_out_train, squared=False)

In [18]:
err_test = sklearn.metrics.mean_squared_error(y_test, y_out_test, squared=False)

In [19]:
err_train

0.20224154171582098

In [20]:
err_test

0.200704280598771

### Further Steps

# A non-linear example

So far we have looked at an example with quite simple, generated data. Now we look at an example that again uses generated data, but now generated through a non-linear process so we can try other regressions algorithms. This is till not a particularly "real world" example, but the algorthms used here are a lot more powerful and would be useful in a real-world problem. What this demonstrates although our regressor is now a lot more powerful, the interfaces for the various bits of our porcessing pipeline looks exactly the same as for our linear case. 


In [22]:
input_nl, target_nl = sklearn.datasets.make_friedman1(n_samples=n_samples, n_features=n_features, noise=noise_std)
input_nl_train, input_nl_test, target_nl_train, target_nl_test = sklearn.model_selection.train_test_split(input_nl, target_nl)
scaler_nl = sklearn.preprocessing.StandardScaler()
scaler_nl.fit(input_nl_train)
X_nl_train = scaler_nl.transform(input_nl_train)
X_nl_test = scaler_nl.transform(input_nl_test)
y_nl_train = target_nl_train
y_nl_test = target_nl_test
reg_mlp1 = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=[20,20,20], max_iter=10000, tol=1e-5)
reg_mlp1.fit(X_nl_train, y_nl_train)
y_nl_out_train = reg_mlp1.predict(X_nl_train)
y_nl_out_test = reg_mlp1.predict(X_nl_test)
err_nl_train = sklearn.metrics.mean_squared_error(y_nl_train, y_nl_out_train, squared=False)
err_nl_test = sklearn.metrics.mean_squared_error(y_nl_test, y_nl_out_test,squared=False)


In [23]:
print(f'RMS error: training data = {err_nl_train:0.3f}; testing data = {err_nl_test:0.3f}')

RMS error: training data = 0.233; testing data = 0.247


## Scikit-learn processing pipeline

More complicated real-world examples of classificat may involve many more steps. Scikit-learn makes this easier by supplying a pipeline class for linking the steps together.